In [ ]:
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import pandas as pd

In [ ]:
source=[]
http_error=[]
attr_error=[]
http_error_source=[]
attr_error_source=[]

In [ ]:
#url to crawl
url_crawl="http://www.logitech.com"

In [ ]:
def getLinks(pageUrl):
    global source_pages
    try:
        html = urlopen(url_crawl+pageUrl)
    except HTTPError as e:
        http_error.append(pageUrl)
        #print(e)
        return None
    try:
        bsObj = BeautifulSoup(html,"lxml")
    except AttributeError as e:
        attr_error.append(pageUrl)
        print(e)
        pass
    for link in bsObj.findAll("a",href=re.compile("^/")):
         if 'href' in link.attrs:
                if link.attrs['href'] not in source_pages:
                    newPage = link.attrs['href']
                    source_pages.append(newPage)
                    getLinks(newPage)
getLinks("")

pd.DataFrame(source_pages).to_csv('source.csv')

In [ ]:
#Retrieves a list of all Internal links found on a page
def getInternalLinks(bsObj, includeUrl):
    includeUrl = urlparse(includeUrl).scheme+"://"+urlparse(includeUrl).netloc
    internalLinks = []
    #Finds all links that begin with a "/"
    for link in bsObj.findAll("a", href=re.compile("^(/|.*"+includeUrl+")")):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in internalLinks:
                if(link.attrs['href'].startswith("/")):
                    internalLinks.append(includeUrl+link.attrs['href'])
                else:
                    internalLinks.append(link.attrs['href'])
    return internalLinks

In [ ]:
links_df=[]
links_df=pd.DataFrame(columns=['source','target'])

In [ ]:
for pp in source_pages:
    source=url_crawl + pp
    try:
        html = urlopen(source)
    except HTTPError as e:
        pass    
    bsObj = BeautifulSoup(html,"lxml")
    int_links_list=getInternalLinks(bsObj, includeUrl=url_crawl)
    for i in range(len(int_links_list)):
        source_temp=source
        target_temp=int_links_list[i]
        temp0=pd.DataFrame({
            'source'       : [source_temp],
            'target'        : [target_temp]})
        links_df=links_df.append(temp0)   

In [ ]:
links_df.to_csv('source_target.csv',index=False)